In [1]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from utils import preprocess_text
from models.transformer import Transformer
from data.collate_fn import collate_fn
from train.lr_scheduler import NoamLR
from train.loss import loss_function
from train.train import train_loop

import torch
from torch.optim import Adam

from data.dataset import AihubTranslationDataset
from torch.utils.data import DataLoader

2.4.0


/home/masang/anaconda3/envs/torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 데이터 전처리

In [3]:
# Dataset
train_ds = AihubTranslationDataset(
    csv_path="ai_hub_dataset/train_filtered.csv",
    preprocess_fn=preprocess_text,
    max_len=60,
    add_special_tokens=True,
)

In [13]:
print(train_ds[1972]["src_ids"])

tensor([    2,  9283,  2470,  3611,  2052,  6263,  2138,  5340,  9253,  9790,
         2073,  3794,  2069,  1889,  2259,  4411,  2119, 15136,  2062,    18,
            3])


In [15]:
from transformers import AutoTokenizer

src_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

print(src_tokenizer.decode(train_ds[1972]["src_ids"]))

[CLS] 현명한 사람이 사기를 당하거나 어리석은 결정을 하는 사례도 흔하다. [SEP]


#### 모델 학습

In [5]:
SRC_VOCAB_SIZE = 32000  # "klue/bert-base" tokenizer vocab_size
TGT_VOCAB_SIZE = 30522  # "bert-base-uncased" tokenizer vocab_size
EPOCHS = 8
BATCH_SIZE = 128
D_MODEL = 256
D_FF = 1024

In [6]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
    collate_fn=collate_fn,
)

In [7]:
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 생성 및 디바이스 이동
model = Transformer(
    src_vocab_size=SRC_VOCAB_SIZE,
    tgt_vocab_size=TGT_VOCAB_SIZE,
    src_len=60,  # 인코더 입력 길이 제한
    tgt_len=60,  # 디코더 입력 길이 제한
    d_model=D_MODEL,
    d_ff=D_FF,
    n_heads=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dropout=0.3,
).to(device)

In [8]:
# optimizer & lr scheduler 정의
optimizer = Adam(model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9)
scheduler = NoamLR(optimizer, d_model=D_MODEL, warmup_steps=4000)

In [9]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_function, optimizer, scheduler, device)
    torch.save(model.state_dict(), "checkpoints/aihub-ko2en-transformer.pt")

Epoch 1
-------------------------------


 17%|█▋        | 1001/5782 [31:39<2:43:50,  2.06s/it]

[train] epoch_end | loss 6.0566 | avg 7.3803


 35%|███▍      | 2001/5782 [1:02:40<2:04:07,  1.97s/it]

[train] epoch_end | loss 5.1696 | avg 6.5086


...

[train] epoch_end | loss 3.1899 | avg 3.1991


 35%|███▍      | 2000/5782 [1:02:02<1:12:58,  1.16s/it]

[train] epoch_end | loss 3.2133 | avg 3.2009


 42%|████▏     | 2448/5782 [1:16:02<1:43:33,  1.86s/it]


KeyboardInterrupt: 

#### 번역문 생성

In [26]:
from inference import get_bleu_score
import pandas as pd

In [27]:
target_df = pd.read_csv("ai_hub_dataset/test_filtered.csv")["번역문"]

In [28]:
print(get_bleu_score(target_df, target_df))

0.9999997417575798
